#1. 파일 읽기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/features.txt', sep='\s+', header=None, names=['index', 'name'])
df.head(3)

,index,name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z


In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/train/X_train.txt', sep='\s+', header=None, names=df.name)
X_train.head(3)

> 실행결과 에러가 발생되는데 이는 feature정보가 중복되어 발생되는 에러이다.

#2. 중복 feature 변경
> feature 이름이 중복되면 파일을 읽어 들일 경우 에러가 발생되게 된다. 이를 해결하기 위해 feature name에 _1, _2 형식으로 이름을 추가할 것이다.

##2.1 중복 확인
> 현재 데이터 프레임에서 중복된 feature에 대해 확인해 보자

In [ ]:
#name으로 그룹화 하여 카운트를 구한다.
df.groupby('name').count()
df_feature_dup = df.groupby('name').count()

#구한 카운트 정보가 index에 저장되어 이 값이 1보다 큰 값, 즉 중복된 값을 구한다.
df_feature_dup[df_feature_dup['index']>1]

In [10]:
df[df['name']== 'fBodyAcc-bandsEnergy()-1,16']

,index,name
310,311,"fBodyAcc-bandsEnergy()-1,16"
324,325,"fBodyAcc-bandsEnergy()-1,16"
338,339,"fBodyAcc-bandsEnergy()-1,16"


> 기존의 df에서 name정보가 'fBodyAcc-bandsEnergy()-1,16'인 값을 확인해 보았더니 위와 같이 3개의 값이 나타나는 것을 볼 수 있다.

In [11]:
dup_list = [310, 324, 338]

> 추후 중복 확인을 위해, 리스트로 목록을 만들었다.

##2.2 중복 변경

In [14]:
df_test = pd.DataFrame([['a'], ['a'], ['a'], ['b'], ['b'], ['c'], ['a'], ['b']], columns=['A'])
df_test

,A
0,a
1,a
2,a
3,b
4,b
5,c
6,a
7,b


In [18]:
df_test['dup_cnt'] = df_test.groupby('A').cumcount()
df_test

,A,dup_cnt
0,a,0
1,a,1
2,a,2
3,b,0
4,b,1
5,c,0
6,a,3
7,b,2


> cumcount는 각 데이터별 중복된 값을 세는 함수 이다.

In [23]:
df_test.apply(lambda x: x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)

0      a
1    a_1
2    a_2
3      b
4    b_1
5      c
6    a_3
7    b_2
dtype: object

##2.3 Qioz
> feature의 중복된 값을 변경하시오

In [26]:
df['dup_cnt'] = df.groupby('name').cumcount()
df

,index,name,dup_cnt
0,1,tBodyAcc-mean()-X,0
1,2,tBodyAcc-mean()-Y,0
2,3,tBodyAcc-mean()-Z,0
3,4,tBodyAcc-std()-X,0
4,5,tBodyAcc-std()-Y,0
...,...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)",0
557,558,"angle(tBodyGyroJerkMean,gravityMean)",0
558,559,"angle(X,gravityMean)",0
559,560,"angle(Y,gravityMean)",0


> name으로 그룹화 시킨 후 cumcount를 이용하여 중복된 값을 count한 후. dup_춧에 저장했다.

In [28]:
df_tmp = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
df_tmp[dup_list]

310      fBodyAcc-bandsEnergy()-1,16
324    fBodyAcc-bandsEnergy()-1,16_1
338    fBodyAcc-bandsEnergy()-1,16_2
dtype: object

> x는 데이터 프레임의 한 행으로 x[0]이 index, x[1]은 name, x[2]는 dup_cnt이다. dup_cnt의 값이 0보다 큰 경우가 중복된 값이 되며, 이럴경우 dup_cnt와 _(언더바)를 붙여 중복을 배제할 수 있다.

> 위의 코드에서는 x[2]가 0보다 큰 경우 x[1]에 _와 dup_cnt를 연결하였고 그렇지 않은 경우 x[1]의 값만 출력하였다.

In [30]:
df.loc[dup_list]

,index,name,dup_cnt
310,311,"fBodyAcc-bandsEnergy()-1,16",0
324,325,"fBodyAcc-bandsEnergy()-1,16",1
338,339,"fBodyAcc-bandsEnergy()-1,16",2


> 이전 코드에서 결과를  df_tmp에 저장했으며 원본 데이터인 df에는 적용되지 않은 것을 볼 수 있다.

In [ ]:
df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
df.loc[dup_list]

> 위의 코드를 여러번 실행하면 name에 _숫자 형식으로 데이터가 붙게 된다. 

In [ ]:
df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
df.drop('dup_cnt', axis=1, inplace=True)
df.loc[dup_list]

> 위 코드는 이전 코드에서 여러번 실행할 경우 발생했던 문제점을 해결하였다. 하지만 이 코드도 여러번 실행하면 dup_cnt를 제거하여 x[2]부분에서 에러가 발생하게된다.

In [32]:
if len(df.columns)>2:
  df['name'] = df.apply(lambda x : x[1]+'_'+str(x[2]) if x[2]>0 else x[1], axis=1)
  df.drop('dup_cnt', axis=1, inplace=True)
df.loc[dup_list]

,index,name
310,311,"fBodyAcc-bandsEnergy()-1,16"
324,325,"fBodyAcc-bandsEnergy()-1,16_1"
338,339,"fBodyAcc-bandsEnergy()-1,16_2"


> 최종적으로 'dup_cnt' feature를 제거했기 떄문에, len(df.columns)의 값은 2가 나와 조건문 내부 코드가 실행되지 않는다. 즉, dup_cnt가 존재하면 중복데이터 변경 후 dup_cnt feature를 제거하고 그렇지 않으면 단순하게 출력만 하는 코드이다.

##2.4 백업하기
> 추후 다른 모델에서 적용할 것이기 떄문에현재 데이터 프레임을 파일로 저장하자


In [33]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/features_dup.txt', header=None)

#3. 데이터 얻어오기
> 훈련 데이터와 테스트 데이터를 파일에서 읽어 결과를 출력하세요

In [47]:
X_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/train/X_train.txt', sep='\s+', header=None, names=df.name)
y_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/train/y_train.txt', sep='\s+', header=None, names=['Action'])

X_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/test/X_test.txt', sep='\s+', header=None, names=df.name)
y_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec6_머신러닝_무작정따라하기/2.의사결정트리/data/test/y_test.txt', sep='\s+', header=None, names=['Action'])
X_train.head(3)

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",...,fBodyBodyAccJerkMag-energy(),fBodyBodyAccJerkMag-iqr(),fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.943220,-0.407747,-0.679338,-0.602122,0.929294,-0.853011,0.359910,-0.058526,0.256892,-0.224848,0.264106,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.660790,...,-0.999918,-0.991364,-1.0,-0.936508,0.346989,-0.516080,-0.802760,-0.980135,-0.961309,-0.973653,-0.952264,-0.989498,-0.980135,-0.999240,-0.992656,-0.701291,-1.000000,-0.128989,0.586156,0.374605,-0.991990,-0.990697,-0.989941,-0.992448,-0.991048,-0.991990,-0.999937,-0.990458,-0.871306,-1.000000,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,-0.557851,-0.818409,0.849308,0.685845,0.822637,-0.981930,-0.999991,-0.999788,-0.998405,-0.999150,-0.977866,-0.948225,-0.714892,-0.500930,-0.570979,0.611627,-0.329549,0.284213,0.284595,0.115705,-0.090963,0.294310,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,0.579382,...,-0.999867,-0.991134,-1.0,-0.841270,0.532061,-0.624871,-0.900160,-0.988296,-0.983322,-0.982659,-0.986321,-0.991829,-0.988296,-0.999811,-0.993979,-0.720683,-0.948718,-0.271958,-0.336310,-0.720015,-0.995854,-0.996399,-0.995442,-0.996866,-0.994440,-0.995854,-0.999981,-0.994544,-1.000000,-1.000000,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,-0.557851,-0.818409,0.843609,0.682401,0.839344,-0.983478,-0.999969,-0.999660,-0.999470,-0.997130,-0.964810,-0.974675,-0.592235,-0.485821,-0.570979,0.273025,-0.086309,0.337202,-0.164739,0.017150,-0.074507,0.342256,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,0.608900,...,-0.999845,-0.986658,-1.0,-0.904762,0.660795,-0.724697,-0.928539,-0.989255,-0.986

In [42]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB


In [48]:
y_train.head(3)

,Action
0,5
1,5
2,5


#4. 모델 만들기

In [73]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=62)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
pred

array([5, 5, 5, ..., 2, 1, 2])

> 위와 같이 모델을 만든 후 검증데이터로 확인해 보면 예측 결과를 확인할 수 있다.

In [70]:
from sklearn.metrics import accuracy_score
print('정확도 : ', accuracy_score(y_test, pred))
print('하이퍼 파라미터 : ', dt_clf.get_params())

정확도 :  0.8591788259246692
하이퍼 파라미터 :  {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 62, 'splitter': 'best'}


#5. 튜닝하기

In [55]:
from sklearn.model_selection import GridSearchCV

def get_best_param(max_depth_list):
  params = {
      'max_depth' : max_depth_list
  }
  grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
  grid_cv.fit(X_train, y_train)

  print('Best score : ', grid_cv.best_score_)
  print('Best Hyper Parameter : ', grid_cv.best_params_)


In [56]:
# 10단위 데이터
#1 부터 30까지 10단위로 증가.
# 1, 11, 21, 31
max_depth_list = range(1, 30, 10)
get_best_param(max_depth_list)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:   40.3s finished


Best score :  0.8475348807095919
Best Hyper Parameter :  {'max_depth': 21}


> 위 코드는 1에서 31까지 10단위로 증가하여 1, 11, 21, 31 총 4개의 파라미터로 결과를 확인한 것이다. 실행 결과 11이 최정의 파라미터로 나타났다. 그러면 11주위의 수가 최적일 가능성이 높다

In [57]:
# 3단위 데이터
#1 부터 30까지 10단위로 증가.
# 11, 14, 17, 20, 23, 26, 29
max_depth_list = range(11, 30, 3)
get_best_param(max_depth_list)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  2.3min finished


Best score :  0.8510720182022503
Best Hyper Parameter :  {'max_depth': 23}


##5.2 두 개의 파라미터 적용

In [74]:
from sklearn.model_selection import GridSearchCV

def get_best_param(max_depth_list, min_samples_split_list):
  params = {
      'max_depth' : max_depth_list,
      'min_samples_split' : min_samples_split_list
  }
  grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
  grid_cv.fit(X_train, y_train)

  print('Best score : ', grid_cv.best_score_)
  print('Best Hyper Parameter : ', grid_cv.best_params_)

In [75]:
# 1단위 데이터
# 20부터 26까지 1단위로 증가.
# 20~26
# 2, 12, 22, 
max_depth_list = range(20, 26, 1)
min_samples_split_list = range(2, 31, 10)
get_best_param(max_depth_list, min_samples_split_list)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  6.1min finished


Best score :  0.8464477402109722
Best Hyper Parameter :  {'max_depth': 21, 'min_samples_split': 2}


# 6. 하이퍼 파라미터를 적용한 모델

In [76]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(max_depth=21, min_samples_split=2, random_state=62)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('정확도 : ', accuracy_score(y_test, pred))

정확도 :  0.8591788259246692
